# Proyecto Final del curso de Ingeniería de Datos 

Se propone crear un pipeline que extraiga datos de una API pública de forma constante combinándolos con información extraída de una base de datos y colocándolos en un Data Warehouse.

## Setup

### Instalación de librerias

In [1]:
# Instalacion de la libreria para interactuar con la base de datos, especificamente con Postgres
#%pip install sqlalchemy psycopg2-binary

### Importación de librerias

In [106]:
# Libreria para interactuar con APIs
import requests

import pandas as pd

# Libreria para interactuar con la base de datos
import sqlalchemy as sa
from configparser import ConfigParser
from pathlib import Path

import psycopg2

### Definición de funciones

In [66]:
def read_api_credentials(config_file: Path, section: str) -> dict:
    """
    Lee las credenciales de la API desdde un archivo de configuracion

    Parametros:
    config_file: Ruta del archivo de configuracion
    section: seccion del archivo de configuracion que contiene las credenciales
    """
    config = ConfigParser()
    config.read(config_file)
    api_credentials = dict(config[section])
    return api_credentials

## Conexion con la API

Extraccion de datos de la API de transporte de Buenos Aires

In [67]:
base_url = "https://apitransporte.buenosaires.gob.ar"

api_keys = read_api_credentials("config/pipeline.conf", "api_transporte")

# No pude con los headers, lo puse como parametros pero oculte la info
params = { 
    "client_id" : api_keys["client_id"],
    "client_secret" : api_keys["client_secret"]
}

In [78]:
# Parametro que a veces es requisitos
formato_json = {'json': 1}

### Extracción de datos de los bus

In [68]:
endpoint_bus = "colectivos"

_____________

Para buses en especifico, si quiero el general no corro ninguna de estas lineas

In [69]:
# LA NUEVA METROPOL S.A.
la_nueva_metropol = {'agency_id': 9}

In [70]:
# MICRO OMNIBUS PRIMERA JUNTA S.A
primera_junta = {'agency_id': 145}

In [132]:
# TRANSPORTE AUTOMOTORES LA PLATA SA
talp = {'agency_id': 155}

______________

#### Información de la posicion de los bus

In [119]:
# Obtencion de la posición de los vehículos monitoreados actualizada cada 30 segundos. 
# Si no se pasan parámetros de entrada, retorna la posición actual de todos los vehículos monitoreados.

endpoint_busPositions = f"{endpoint_bus}/vehiclePositionsSimple"

full_url_busPositions = f"{base_url}/{endpoint_busPositions}"


##### Para acceder a las posiciones de las lineas de Primera Junta

In [121]:
params_PJPositions = params.copy()
params_PJPositions.update(primera_junta)

In [122]:
r_PJPositions = requests.get(full_url_busPositions, params=params_PJPositions)

r_PJPositions.status_code

200

In [125]:
json_PJData = r_PJPositions.json()
json_PJData

[{'route_id': '1293',
  'latitude': -34.80142,
  'longitude': -58.2553,
  'speed': 0,
  'timestamp': 1707258812,
  'id': '23696',
  'direction': 0,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R9',
  'tip_id': '83133-1',
  'trip_headsign': 'a Moreno x Panamericana'},
 {'route_id': '1279',
  'latitude': -34.73407,
  'longitude': -58.26285,
  'speed': 0,
  'timestamp': 1707258814,
  'id': '23697',
  'direction': 0,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R3',
  'tip_id': '82347-1',
  'trip_headsign': 'A - Barrio Sitra - IDA'},
 {'route_id': '1297',
  'latitude': -34.7516,
  'longitude': -58.2678947,
  'speed': 6.111111,
  'timestamp': 1707258814,
  'id': '23726',
  'direction': 0,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R16',
  'tip_id': '83377-1',
  'trip_headsign': 'Ramal 13 - IDA'},
 {'route_id': '1285',
  'latitude': 

In [126]:
type(json_PJData)

list

In [127]:
json_PJData[1].keys()

dict_keys(['route_id', 'latitude', 'longitude', 'speed', 'timestamp', 'id', 'direction', 'agency_name', 'agency_id', 'route_short_name', 'tip_id', 'trip_headsign'])

Para pasar a un dataframe la data


In [128]:
# Para pasar el json a una dataframe
df_PJPositions = pd.json_normalize(json_PJData)
df_PJPositions.sample(n=10)

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
40,1295,-34.81446,-58.272500,7.500000,1707258812,23943,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9F,83249-1,a Pilar x Ford
59,1289,-34.83154,-58.211720,21.944445,1707258838,24407,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R6P,82939-1,Ramal B - a A. Bello
51,1294,-34.83204,-58.231810,5.277777,1707258814,24038,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9,83181-1,a Pte. Saavedra
64,1295,-34.81183,-58.272640,1.111111,1707258812,26365,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9F,83249-1,a Pilar x Ford
38,1279,-34.72050,-58.262400,0.000000,1707258812,23941,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R3,82346-1,A - Barrio Sitra - IDA
33,1277,-34.86600,-58.191160,2.500000,1707258812,23922,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R2F,82248-1,a Boulogne - S.Isidro
60,1285,-34.85678,-58.212975,2.500000,1707258782,24445,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324T5,82692-1,B - Barrio Centenario (por Milan) - IDA
46,1290,-34.77314,-58.281975,15.000000,1707258814,23972,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R6P,82992-1,Ramal B - a Est. Lomas de Zamora
45,1286,-34.77994,-58.260994,8.333333,1707258782,23966,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324T5,82827-1,B - Barrio Centenario (por Milan) - VUELTA
7,1280,-34.82992,-58.183580,0.000000,1707258812,23768,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R3,82394-1,a Tribunales de Retiro/Htal. Ferroviario


##### Para acceder a las posiciones de las lineas de La Nueva Metropol

In [129]:
params_NMPositions = params.copy()
params_NMPositions.update(la_nueva_metropol)
r_NMPositions = requests.get(full_url_busPositions, params=params_NMPositions)

r_NMPositions.status_code

200

In [130]:
json_NMData = r_NMPositions.json()
df_NMPositions = pd.json_normalize(json_NMData)
df_NMPositions.sample(n=10)

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
173,1203,-34.51999,-58.753880,3.055555,1707258992,24840,1,LA NUEVA METROPOL S.A.,9,365R6,77832-1,a Los Pinos
167,2038,-34.57116,-58.442380,0.000000,1707258992,21358,0,LA NUEVA METROPOL S.A.,9,194H,140363-1,a Est. Escobar
62,2038,-34.47589,-58.659794,3.611111,1707258992,8430,0,LA NUEVA METROPOL S.A.,9,194H,140352-1,a Est. Escobar
4,2007,-34.56357,-58.455395,7.777777,1707259020,5962,0,LA NUEVA METROPOL S.A.,9,65A,137199-1,a Est. Avellaneda
34,141,-34.60940,-58.369400,7.500000,1707259020,8006,1,LA NUEVA METROPOL S.A.,9,195C,11989-1,a Almirante Brown - VUELTA
112,1200,-34.54504,-58.808106,5.000000,1707258992,20441,0,LA NUEVA METROPOL S.A.,9,365R5,77682-1,a Los Pinos
129,1200,-34.56824,-58.808834,0.000000,1707258992,20572,0,LA NUEVA METROPOL S.A.,9,365R5,77681-1,a Los Pinos
38,2042,-34.59885,-58.403934,1.388888,1707258990,8286,0,LA NUEVA METROPOL S.A.,9,194A,140594-1,Ramal A - IDA
196,1595,-34.32002,-58.863120,26.944445,1707259020,50039,1,LA NUEVA METROPOL S.A.,9,194E,101886-1,Expreso - Pza. Miserere
65,2007,-34.56205,-58.456314,1.666666,1707258992,8878,0,LA NUEVA METROPOL S.A.,9,65A,137197-1,a Est. Avellaneda


##### Para acceder a las posiciones de las lineas TALP

In [133]:
params_TALPPositions = params.copy()
params_TALPPositions.update(talp)
r_TALPPositions = requests.get(full_url_busPositions, params=params_TALPPositions)

r_TALPPositions.status_code

200

In [134]:
json_TALPData = r_TALPPositions.json()
df_TALPPositions = pd.json_normalize(json_TALPData)
df_TALPPositions.sample(n=10)

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
19,1250,-34.68608,-58.559506,0.000000,1707259292,23813,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338C,80574-1,Ramal F - RN 3 x Alberdi
7,1247,-34.51061,-58.566720,0.000000,1707259322,23702,1,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338B,80505-1,a Ituzaingo y 29 de Septiembre
10,1250,-34.64922,-58.618750,4.722222,1707259322,23749,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338C,80574-1,Ramal F - RN 3 x Alberdi
0,1250,-34.75620,-58.484245,0.000000,1707259292,20143,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338C,80577-1,Ramal F - RN 3 x Alberdi
28,1247,-34.93938,-57.938980,0.000000,1707259292,23929,1,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338B,80490-1,a Ituzaingo y 29 de Septiembre
27,1250,-34.53919,-58.578650,0.000000,1707259292,23919,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338C,80572-1,Ramal F - RN 3 x Alberdi
16,1255,-34.63989,-58.556885,0.000000,1707259322,23786,1,TRANSPORTE AUTOMOTORES LA PLATA SA,155,406A,80737-1,a Cement. de Villegas x Mocoreta
11,1254,-34.75826,-58.482960,10.833333,1707259292,23750,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,406A,80691-1,a B° Central x Mocoreta
12,1247,-34.78049,-58.263340,0.000000,1707259264,23752,1,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338B,80494-1,a Ituzaingo y 29 de Septiembre
4,1250,-34.70637,-58.532875,7.500000,1707259294,23652,0,TRANSPORTE AUTOMOTORES LA PLATA SA,155,338C,80577-1,Ramal F - RN 3 x Alberdi


#### Informacion de servicios de alerta

In [144]:
# Las alertas de servicio te permiten proporcionar actualizaciones cada vez que se produce una interrupción en la red. 
# Las demoras y cancelaciones de viajes individuales a menudo se deben comunicar a través de los trip updates.

endpoint_busServiceAlerts = f"{endpoint_bus}/serviceAlerts"

full_url_busServiceAlerts = f"{base_url}/{endpoint_busServiceAlerts}"

params_busServiceAlerts = params.copy()
params_busServiceAlerts.update(formato_json)



In [145]:
r_busServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_busServiceAlerts)

r_busServiceAlerts.status_code

200

In [146]:
json_busData = r_busServiceAlerts.json()
json_busData

{'_entity': [{'_alert': {'_active_period': [],
    '_cause': 2,
    '_description_text': {'_translation': [{'_language': '',
       '_text': 'Por desvío, la linea VERDE se detiene acá',
       'extensionObject': None}],
     'extensionObject': None},
    '_effect': 4,
    '_header_text': {'_translation': [{'_language': '',
       '_text': 'Paradas provisorias de la línea VERDE',
       'extensionObject': None}],
     'extensionObject': None},
    '_informed_entity': [{'_agency_id': '',
      '_route_id': None,
      '_route_type': 0,
      '_stop_id': '73001',
      '_trip': None,
      'extensionObject': None},
     {'_agency_id': '',
      '_route_id': None,
      '_route_type': 0,
      '_stop_id': '15011',
      '_trip': None,
      'extensionObject': None}],
    '_url': {'_translation': [], 'extensionObject': None},
    'extensionObject': None},
   '_id': '5274244',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': None,
   'extensionObject': None},
  {'_alert': {'_

##### Para avisos de Primera Junta

In [152]:
params_PJServiceAlerts = params_busServiceAlerts.copy()

params_PJServiceAlerts.update(primera_junta)

r_PJServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_PJServiceAlerts)

r_PJServiceAlerts.status_code

200

In [153]:
json_PJServiceAlerts = r_PJServiceAlerts.json()
json_PJServiceAlerts.keys()

dict_keys(['_entity', '_header', 'extensionObject'])

In [154]:
data_PJServiceAlerts = json_PJServiceAlerts['_entity']
df_PJServiceAlerts = pd.DataFrame(data_PJServiceAlerts)
df_PJServiceAlerts

""


##### Para avisos de La Nueva Metropol

In [155]:
params_NMServiceAlerts = params_busServiceAlerts.copy()

params_NMServiceAlerts.update(la_nueva_metropol)

r_NMServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_NMServiceAlerts)

r_NMServiceAlerts.status_code

200

In [157]:
json_NMServiceAlerts = r_NMServiceAlerts.json()
data_NMServiceAlerts = json_NMServiceAlerts['_entity']
df_NMServiceAlerts = pd.DataFrame(data_NMServiceAlerts)
df_NMServiceAlerts

""


##### Para avisos de TALP

In [158]:
params_TALPServiceAlerts = params_busServiceAlerts.copy()

params_TALPServiceAlerts.update(talp)

r_TALPServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_TALPServiceAlerts)

r_TALPServiceAlerts.status_code

200

In [159]:
json_TALPServiceAlerts = r_TALPServiceAlerts.json()
data_TALPServiceAlerts = json_TALPServiceAlerts['_entity']
df_TALPServiceAlerts = pd.DataFrame(data_TALPServiceAlerts)
df_TALPServiceAlerts

""


### Extracción de datos del estado de las estaciones de las ecobicis

In [47]:
# Obtencion del número de bicicletas y anclajes disponibles en cada estación y disponibilidad de estación.

endpoint_ecobiciSS = "ecobici/gbfs/stationStatus"
full_url_ecobiciSS = f"{base_url}/{endpoint_ecobiciSS}"

r_ecobiciSS = requests.get(full_url_ecobiciSS, params=params)

In [48]:
r_ecobiciSS.status_code

200

In [49]:
json_ecobiciSS = r_ecobiciSS.json()
json_ecobiciSS

{'last_updated': 1707186551,
 'ttl': 8,
 'data': {'stations': [{'station_id': '2',
    'num_bikes_available': 13,
    'num_bikes_available_types': {'mechanical': 13, 'ebike': 0},
    'num_bikes_disabled': 2,
    'num_docks_available': 25,
    'num_docks_disabled': 0,
    'last_reported': 1707186544,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '3',
    'num_bikes_available': 5,
    'num_bikes_available_types': {'mechanical': 5, 'ebike': 0},
    'num_bikes_disabled': 2,
    'num_docks_available': 21,
    'num_docks_disabled': 0,
    'last_reported': 1707186340,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '4',
    'num_bikes_available': 3,
    'num_bikes_available_types': {'mechanical': 3, 'ebike': 0},
    'num_bikes_disabled': 0,
    'num_doc

In [50]:
json_ecobiciSS.keys()

dict_keys(['last_updated', 'ttl', 'data'])

In [51]:
# Para pasar el json a una dataframe

data_ecobiciSS= json_ecobiciSS['data']['stations']
df_ecobiciSS = pd.DataFrame(data_ecobiciSS)

df_ecobiciSS.sample(n=10)

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic
284,434,1,"{'mechanical': 1, 'ebike': 0}",0,19,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
256,385,7,"{'mechanical': 7, 'ebike': 0}",0,9,0,1.707187e+09,False,IN_SERVICE,1,1,1,None
290,446,0,"{'mechanical': 0, 'ebike': 0}",0,0,0,NaN,False,END_OF_LIFE,1,0,0,None
45,65,3,"{'mechanical': 3, 'ebike': 0}",1,16,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
124,181,1,"{'mechanical': 1, 'ebike': 0}",4,15,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
82,117,4,"{'mechanical': 4, 'ebike': 0}",2,14,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
81,116,1,"{'mechanical': 1, 'ebike': 0}",1,10,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
79,112,1,"{'mechanical': 1, 'ebike': 0}",2,17,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
207,311,8,"{'mechanical': 8, 'ebike': 0}",0,12,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
148,213,1,"{'mechanical': 1, 'ebike': 0}",2,17,0,1.707186e+09,False,IN_SERVICE,1,1,1,None


## Conexión con base de datos

In [115]:
db_keys = read_api_credentials("config/pipeline.conf", "RedShift")

try:
    conn = psycopg2.connect(
        host = db_keys["host"],
        dbname = db_keys["dbname"] ,
        user = db_keys['user'],
        password = db_keys['pwd'],
        port = db_keys["port"],
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


### Tablas para datos de los bus

Tabla para las agencias de interes

In [109]:
with conn.cursor() as cur:
    cur.execute("""
        create table if not exists  camilagonzalezalejo02_coderhouse.agencies
        (       	
	    agency_id INTEGER,
	    agency_name VARCHAR(100)
        )
    sortkey(agency_id)
    """)
    conn.commit()

Tabla para los viajes realizados por dichas agencias

In [117]:
try:
    with conn.cursor() as cur:
        cur.execute("""
            DROP TABLE positions;
            create table if not exists  camilagonzalezalejo02_coderhouse.positions
            (	
            id INTEGER,
            agency_id INTEGER,
            route_id INTEGER,
            latitude NUMERIC,
            longitude NUMERIC,
            speed NUMERIC,
            route_short_name VARCHAR(50),
            trip_headsign VARCHAR(100),
            alert VARCHAR(100),
            date_and_time timestamp 
            )   
        """)
        conn.commit()
except psycopg2.Error as e:
    print("Error al ejecutar la consulta SQL:", e)